### Architecture

Pytorch

- input layer
- some kind of deep averaging net
- last_shared_layer = torch.xxx()

classification specific net:
first_layer_class_output = torch.xxx(last_shared_layer)
...
last layer = softmax that outputs 5 labels


sentiment specific net:
first_layer_sent_output = torch.xxx(last_shared_layer)
...
last layer = softmax that outputs 3 labels

forward() returns 2 elements?                                                                                                                                                                                                                                                                         

In [115]:
# Forward pass 
# TODO: MAKE THIS WORK-- we have a skeleton of how we want it to work here, fill in the rest
pred_class, pred_sent = self.forward(X_batch)
loss = loss_fn1(pred_class, Y_batch[0]) +loss_fn2(pred_sent, Y_batch[1]) 

NameError: name 'self' is not defined

# Model Pipeline

In [209]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sklearn 
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from gensim.models.keyedvectors import KeyedVectors
import matplotlib.pyplot as plt
import nltk

In [210]:
def load_dataset():

    train = pd.read_csv("./dataset_bbc/train_w_sentiment.csv")
    test = pd.read_csv("./dataset_bbc/test_w_sentiment.csv")
    test_category = pd.read_csv("./dataset_bbc/sample_solution.csv")

    test["Category"] = test_category["Category"]

    Xmat = train["Text"]
    Y = train[["sentiment","Category"]]

    Xmat_test = test["Text"]
    Y_test = test[["sentiment","Category"]]
    
    return Xmat, Y, Xmat_test, Y_test

In [211]:
Xmat, Y, Xmat_test, Y_test = load_dataset()
Xmat_train, Xmat_dev, Y_train, Y_dev = train_test_split(Xmat, Y, test_size=0.3, random_state=42)

In [212]:
# Tokenize
def tokenizer(text: str):
    '''
    Citation: This tokenizer function & regex rule is borrowed from Katie's tokenizer regex demo at:
    https://www.cs.williams.edu/~kkeith/teaching/s23/cs375/attach/tokenization_regex_demo.html
    This helper function takes a string and returns a list of tokenized strings.
    '''
    regex = r"[A-Za-z]+|\$[\d\.]+|\S+" 
    res = nltk.regexp_tokenize(text, regex)
    return [i for i in res if i != "."]

In [213]:
# Embedding
def load_embeddings(filename):
    """
    Loads embeddings from embedding file and creates 
    1) dictionary of embedding words to indices
    2) list of embedding indices to words
    3) dense word embedding matrix
    """
    embeddings = KeyedVectors.load_word2vec_format(filename, binary=False)
    vocab2indx = dict(embeddings.key_to_index)
    idx2vocab = list(embeddings.index_to_key)
    embed_array = embeddings.vectors # matrix of dense word embeddings 
                                     # rows: a word 
                                     # columns: dimensions (50) of the dense embeddings
    return vocab2indx, idx2vocab, embed_array


def add_the_embedding(embed_array, vocab2indx): 
    """
    Adds "the" embedding to the embed_array matrix
    """
    the_embedding = embed_array[vocab2indx["the"]]
    out = np.vstack((embed_array, the_embedding))
    return out


def add_oov(idx2vocab, vocab2indx, embed_array):
    """
    Adds <OOV> token to embedded vocabulary
    """
    print("len embed array: ", len(embed_array))
    new_oov_entry = len(embed_array)
    idx2vocab += ["<OOV>"]
    vocab2indx["<OOV>"] = new_oov_entry
    embed_array_w_oov = add_the_embedding(embed_array, vocab2indx)

    return idx2vocab, vocab2indx, embed_array_w_oov


def add_pad(idx2vocab, vocab2indx, embed_array):
    """
    Adds <PAD> token to embedded vocabulary
    """
    print("len embed array: ", len(embed_array))
    new_pad_entry = len(embed_array)
    idx2vocab += ["<PAD>"]
    vocab2indx["<PAD>"] = new_pad_entry
    embed_array_w_pad = add_the_embedding(embed_array, vocab2indx)
    
    return idx2vocab, vocab2indx, embed_array_w_pad

In [227]:
# Truncate and pad

def truncate(original_indices_list: list, maximum_length=100) -> list: 
    """
    Truncates the original_indices_list to the maximum_length
    """
    return original_indices_list[0:maximum_length]


def pad(original_indices_list: list, pad_index: int, maximum_length=100) -> list: 
    """
    Given original_indices_list, concatenates the pad_index enough times 
    to make the list to maximum_length. 
    """
    while len(original_indices_list) < maximum_length:
        original_indices_list.append(pad_index)
        
    return original_indices_list


def get_padded_oov_embeddings():
    """
    Get embedding array which includes the <PAD> and <OOV> tokens
    """
    vocab2indx, idx2vocab, embed_array = load_embeddings("glove50_4k.txt")
    idx2vocab, vocab2indx, embed_array_w_oov = add_oov(idx2vocab, vocab2indx, embed_array)
    idx2vocab, vocab2indx, embed_array_w_oov_pad = add_pad(idx2vocab, vocab2indx, embed_array_w_oov)
    
    return embed_array_w_oov_pad, vocab2indx, idx2vocab

In [228]:
def create_word_indices(tokens, vocab2indx): 
    """
    For each example, translate each token into its corresponding index from vocab2indx
    
    Replace words not in the vocabulary with the symbol "<OOV>" 
        which stands for 'out of vocabulary'
        
    Arguments: 
       - tokens (List[str]): list of strings of tokens 
       - vocab2indx (dict): each vocabulary word as strings and its corresponding int index 
                           for the embeddings 
                           
    Returns: 
        - (List[int]): list of integers
    """
    indices = []

    for token in tokens:
        if token not in vocab2indx:
            token = "<OOV>"
        indices.append(vocab2indx[token])
    
    return indices

In [229]:
#Unit test 1
toks1 = "this is great".split()
out1 = create_word_indices(toks1, vocab2indx)
out1
train1_toks = Xmat[0]
print("Training example 1 tokens = ", train1_toks)

Training example 1 tokens =  worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practi

In [238]:
def convert_X(Xmat):
    MAXIMUM_LENGTH = 512
    embeddings, vocab2indx, idx2vocab = get_padded_oov_embeddings()
    
    X_list = []
    for one_train_example in Xmat: 
        one_train_example = tokenizer(one_train_example)
        one_train_indices = create_word_indices(one_train_example, vocab2indx)
        one_train_indices = truncate(one_train_indices, maximum_length=MAXIMUM_LENGTH)
        one_train_indices = pad(one_train_indices, len(vocab2indx)-1, maximum_length=MAXIMUM_LENGTH)
        
        one_train_example_embeddings = [] # A list of token embeddings
        
        for index in one_train_indices:
            one_train_example_embeddings.append(embeddings[index])
        
        X_list.append(one_train_example_embeddings)
        
    X = torch.FloatTensor(X_list)
    return X

In [239]:
X = convert_X(Xmat_train)

len embed array:  4196
len embed array:  4197


In [240]:
print(X.shape)

torch.Size([1043, 512, 50])


In [241]:
tokenizer(Xmat_train[0])

['worldcom',
 'ex',
 '-boss',
 'launches',
 'defence',
 'lawyers',
 'defending',
 'former',
 'worldcom',
 'chief',
 'bernie',
 'ebbers',
 'against',
 'a',
 'battery',
 'of',
 'fraud',
 'charges',
 'have',
 'called',
 'a',
 'company',
 'whistleblower',
 'as',
 'their',
 'first',
 'witness',
 'cynthia',
 'cooper',
 'worldcom',
 's',
 'ex',
 '-head',
 'of',
 'internal',
 'accounting',
 'alerted',
 'directors',
 'to',
 'irregular',
 'accounting',
 'practices',
 'at',
 'the',
 'us',
 'telecoms',
 'giant',
 'in',
 '2002.',
 'her',
 'warnings',
 'led',
 'to',
 'the',
 'collapse',
 'of',
 'the',
 'firm',
 'following',
 'the',
 'discovery',
 'of',
 'an',
 '$11',
 'bn',
 '(£5.7bn)',
 'accounting',
 'fraud',
 'mr',
 'ebbers',
 'has',
 'pleaded',
 'not',
 'guilty',
 'to',
 'charges',
 'of',
 'fraud',
 'and',
 'conspiracy',
 'prosecution',
 'lawyers',
 'have',
 'argued',
 'that',
 'mr',
 'ebbers',
 'orchestrated',
 'a',
 'series',
 'of',
 'accounting',
 'tricks',
 'at',
 'worldcom',
 'ordering',
 '

In [245]:
def proba_to_label(proba_batch):

    # Find the index of the largest value in each sub-array
    max_indices = np.argmax(proba_batch, axis=1)

    # Create a new array of the same shape filled with 0s
    binary_array = np.zeros_like(proba_batch)

    # Set the largest value positions to 1
    for i, max_index in enumerate(max_indices):
        binary_array[i, max_index] = 1
    
    return binary_array

In [246]:
proba = [[0.1, 0.7, 0.3],[0.9, 0.05, 0.05]]
proba_to_label(proba)

array([[0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
# Build architecture

# Distilled Dual-task Deep Averaging Net
class DistilledDualDan(nn.Module):
    """
    Pytorch implementation for Deep Averaging Network for classification 
    """
    def __init__(self, num_classes_sentiment, 
                       num_classes_classification,
                       embedding_dim: int, #architecture/pre-processing decision 
                       hidden_dim1: int, #architecture decision
                       hidden_dim2: int, #architecture decision 
                       leaky_relu_negative_slope: float, #hyperparameter
                       dropout_probability: float #hyperparameter
                ):
        """
        Create the network architecture. 
        
        Hints: 
        - Make sure all your dimensions of various layers work out correctly 
        """
        super().__init__()
        self.num_classes_sentiment = num_classes_sentiment
        self.num_classes_classification = num_classes_classification
        
        self.embedding_dim = embedding_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.leaky_relu_negative_slope = leaky_relu_negative_slope
        self.dropout_probability = dropout_probability
        
        # Shared layers between the 2 tasks
        self.hidden1 = nn.Linear(self.embedding_dim, self.hidden_dim1)
        self.hidden2 = nn.Linear(self.hidden_dim1,self.hidden_dim2)
        
        # Sentiment-specific layers
        self.theta_sentiment = nn.Linear(self.hidden_dim2, self.num_classes_sentiment)
        
        # Classification-specific layers
        self.theta_classification = nn.Linear(self.hidden_dim2, self.num_classes_classification)
        
        # Common "tools"
        self.log_softmax = nn.LogSoftmax(dim=1) # A dimension along which LogSoftmax will be computed.
        self.apply_dropout = nn.Dropout(self.dropout_probability)
        
        
    def forward(self, X_batch: torch.Tensor) -> torch.Tensor:
        """
        Given X_batch, make the forward pass through the network. 
        
        The output should be the predicted *log probabilities*. 
        
        Returns: 
            - (torch.Tensor): the log probabilites after the forward pass 
                The shape of this tensor should be (X_batch.shape[0], 2)
                
        Hints: 
            - Look at Pytorch's implemenation of .mean()
            - There should be NO for-loops in this method
        """
        # activation function: leaky_relu
        # probability function used on the forward pass = softmax w log applied
        # leaky_relu_forward_backward(u:float, negative_slope: float=0.1)
        
        # for each example, get mean of word embeddings on the row (dim=1)
            # expected dimensionality: [m x d], where m is the num of examples 
            # (2, in the toy example) and d is the number of dimensions (50, 
            # in the toy example)
        
        # convert from inputted word indices to corresponding vector embeddings
        
        # embedded_with_dropout = self.apply_dropout(X_bat)
        
        # pool input by average so embedding shape goes from (# examples, # words, embedding dimension)
        # to (# examples, embedding dimension)
        
        # TODO - Dropout!
        
        # Common layers
        pooled_input_batch = torch.mean(X_batch, dim=1)
        hid1_output_batch = self.hidden1(pooled_input_batch)
        hid1_output_batch = torch.nn.functional.leaky_relu(hid1_output_batch, self.leaky_relu_negative_slope)
        
        hid2_output_batch = self.hidden2(hid1_output_batch)
        hid2_output_batch = torch.nn.functional.leaky_relu(hid2_output_batch, self.leaky_relu_negative_slope)
        
        # Sentiment-specific layers
        out_sentiment_batch = torch.FloatTensor(self.theta_sentiment(hid2_output_batch))
        log_probs_sentiment_batch = self.log_softmax(out_sentiment_batch)
        
        # Classification-specific layers
        
        out_classification_batch = torch.FloatTensor(self.theta_classification(hid2_output_batch))
        log_probs_classification_batch = self.log_softmax(out_classification_batch)
        
        return log_probs_sentiment_batch, log_probs_classification_batch
        
    def train_model(self, 
                    X_train, 
                    Y_train, # need to be one-hot encoding
                    X_dev, 
                    Y_dev, # need to be one-hot encoding
                    sentiment_soft_labels,
                    classification_soft_labels,
                    soft_label_weight=0.5,
                    loss_fn=nn.CrossEntropyLoss(),
                    optimizer, 
                    num_iterations, 
                    batch_size = 500, 
                    check_every=10, 
                    verbose=False): 
        """
        Method to train the model. 
        
        No need to modify this method. 
        """
        self.train() # tells nn.Module its in training mode 
                      # (important when we get to things like dropout)
            
        loss_history = [] # We'll record the loss for inspection
        train_accuracy = []
        dev_accuracy = []
        
        for t in range(num_iterations):
            
            # Randomly selecting a batch
            if batch_size >= X_train.shape[0]: 
                X_batch = X_train
                Y_batch = Y_train
                sentiment_soft_labels_batch = sentiment_soft_labels
                classification_soft_labels_batch = classification_soft_labels
            else: #randomly choose batch_size number of examples 
                batch_indices = np.random.randint(X_train.shape[0], size=batch_size)
                X_batch = X_train[batch_indices]
                Y_batch = Y_train[batch_indices]
                sentiment_soft_labels_batch = sentiment_soft_labels[batch_indices]
                classification_soft_labels_batch = classification_soft_labels[batch_indices]
          
            # TODO!!! CHECK THIS!!!
            Y_sentiment_batch = Y_batch[1]
            Y_classification_batch = Y_batch[0]

            # Forward pass 
            log_probs_sentiment_batch, log_probs_classification_batch = self.forward(X_batch)

            # distillation loss (cross entropy loss with hard labels + cross entropy loss with soft labels)
            # weighted with soft and hard label
            sentiment_loss = (1-soft_label_weight)*loss_fn(log_probs_sentiment_batch, Y_sentiment_batch) + 
                             soft_label_weight*loss_fn(log_probs_sentiment_batch, sentiment_soft_labels_batch)
                
            classification_loss = (1-soft_label_weight)*loss_fn(log_probs_classification_batch, Y_classification_batch) + 
                             soft_label_weight*loss_fn(log_probs_classification_batch, classification_soft_labels_batch)
            
            # Question: divided by batch size?
            total_loss = sentiment_loss + classification_loss

            #Backprop
            optimizer.zero_grad() # clears the gradients from the previous iteration
                                  # this step is important because otherwise Pytorch will 
                                  # *accumulate* gradients for all itereations (all backwards passes)
            loss.backward() # calculate gradients from forward step 
            optimizer.step() # gradient descent update equation 

            #Check the loss and train and dev accuracies every 
            if t % check_every == 0:
                loss_value = loss.item() # call .item() to detach from the tensor 
                loss_history.append(loss_value)

                #Check train accuracy (entire set, not just batch) 
                train_y_pred, _ = self.predict(X_train)
                train_acc = self.accuracy(train_y_pred, Y_train.detach().numpy()) 
                train_accuracy.append(train_acc)

                #Check dev accuracy (entire set, not just batch) 
                dev_y_pred, _ = self.predict(X_dev)
                dev_acc = self.accuracy(dev_y_pred, Y_dev.detach().numpy())
                dev_accuracy.append(dev_acc)

                if verbose: print(f"Iteration={t}, Loss={loss_value}")
                
        return loss_history, train_accuracy, dev_accuracy
    
    def predict(self, X, proba_mode=False):
        """
        Method to make predictions given a trained model. 
        
        No need to modify this method. 
        """
        self.eval() # tells nn.Module its NOT in training mode 
                 # (important when we get to things like dropout)
    
        log_probs_sentiment_batch, log_probs_classification_batch = self.forward(X)
        
        if proba_mode:
            return log_probs_sentiment_batch, log_probs_classification_batch
        else:
            # convert proba to label
            label_sentiment_batch = proba_to_label(log_probs_sentiment_batch)
            label_classification_batch = proba_to_label(log_probs_classification_batch)
            
        
        if self.num_classes == 2: 
            log_pred_pos_class = pred_log_probs[:,1].detach().numpy() #get only the positive class 
            pred_pos_class = np.exp(log_pred_pos_class) #exp to undo the log 
            # decision threshold
            y_pred = np.zeros(X.shape[0])
            y_pred[pred_pos_class>= 0.5] = 1
            return y_pred, pred_pos_class
        
        else: 
            return pred_log_probs
    
    @staticmethod
    def accuracy(y_pred: np.ndarray, y_true: np.ndarray) -> float: 
        """
        Calculates accuracy. No need to modify this method. 
        """
        return np.mean(y_pred == y_true)

In [ ]:
# Data train-dev-test split
# a pipeline for hyperparameter tuning and training

In [ ]:
# Create baseline